A weighted average of a players stats over the past three seasons can be used to predict next season's perfomance, whereby more recent seasons are weighted more heavily.

In [2]:
import utils
import os
import pandas as pd
from scipy.stats import zscore 
from tqdm.notebook import tqdm
from constants import DATA_DIR

In [3]:
def add_player_value(df):
    #calculate zscore - blocks not included!!
    #consider punting more than one
    columns = ['FG%','FT%','3P','PTS','TRB','AST','STL']
    numeric_stats = df[columns]
    z_scores = pd.DataFrame(zscore(numeric_stats, axis=0, nan_policy='omit'),columns=columns)
    value = pd.DataFrame(columns = ['VALUE'])
    value['VALUE'] = z_scores.sum(axis=1)
    result = pd.concat([df, value], axis=1)
    return result

In [4]:
def weigh_rows(row, weighting, old_row):
    if weighting == 'linear':
        weights = [i for i in range(1,4)]
    elif weighting == 'quad':
        weights = [i**2 for i in range(1,4)]
    elif weighting == 'sqrt':
        weights = [i**(1/2) for i in range(1,4)]
    elif weighting == 'base':
        weights = [1, 1, 1]
    new_row = pd.DataFrame(columns=row.columns)
    new_row.drop(['PLAYER','POS','AGE','TEAM','SEASON','VALUE'],axis=1,inplace=True)
    for column in new_row.columns:
        values = row[column].tolist()
        n = len(weights)
        weighted_sum = [weights[i]*values[i] for i in range(n)]
        weighted_mean = sum(weighted_sum)/sum(weights)
        old_row.loc[:,column] = weighted_mean
    return old_row

In [13]:
def weigh_data(weighting, season, merged, yeet=True):
    curr = merged.loc[merged['SEASON']==season].reset_index()
    weighted = pd.DataFrame()
    for i in tqdm(range(curr.shape[0])): #iterate through players
        name = curr.loc[i,'PLAYER']
        df_name = merged.loc[merged['PLAYER']==name].reset_index(drop=True)
        if yeet == True:
            new = df_name.loc[df_name['SEASON'] < season]
        else:
            new = df_name.loc[df_name['SEASON'] <= season]
        if len(new.index) >=3:
            new.sort_values(by=['SEASON'], inplace=True)
            row = new.iloc[-3:]
            old_row = df_name.loc[df_name['SEASON'] == season]
            new_row = weigh_rows(row, weighting, old_row)
            # add value to row
            weighted = weighted.append(new_row)
    weighted = weighted.fillna(0)
    return weighted

In [6]:
# combine data
seasons = ['2009-10','2010-11','2011-12','2012-13','2013-14','2014-15', 
           '2015-16','2016-17','2017-18','2018-19','2019-20','2020-21']
merged = pd.DataFrame()
for season in seasons:
    df = pd.read_csv('~/Desktop/NBA-Fantasy-Draft/data/PlayerStats/'+season+'.csv')
    df = add_player_value(df)
    df.to_csv('~/Desktop/NBA-Fantasy-Draft/data/Value/'+season+'.csv')
merged = utils.csv_concatenate(os.path.join(DATA_DIR,'Value'))
merged = merged.iloc[:,1:]

In [7]:
weightings = ['base', 'linear', 'quad', 'sqrt']
seasons = ['2012-13','2013-14','2014-15','2015-16','2016-17','2017-18','2018-19','2019-20','2020-21']
for weighting in tqdm(weightings):
    for season in seasons:
        df = weigh_data(weighting, season, merged)
        df.to_csv(os.path.join(DATA_DIR,'Weighted',weighting,season+'.csv'), index=False)

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Create test dataset

In [16]:
weighting = 'sqrt'
season = '2020-21'
df = weigh_data(weighting, season, merged, False)
df.to_csv(os.path.join(DATA_DIR,'Predictions/predictions.csv'), index=False)

              PLAYER POS  AGE TEAM   G  GS    MP   FG   FGA    FG%  ...  DRB  \
2  Bogdan Bogdanović  SG   26  SAC  70  17  27.8  5.2  12.3  0.418  ...  2.9   
0  Bogdan Bogdanović  SG   27  SAC  61  28  29.0  5.5  12.6  0.440  ...  3.0   
3  Bogdan Bogdanović  SG   28  ATL  44  27  29.7  6.1  12.8  0.473  ...  3.2   

   TRB  AST  STL  BLK  TOV   PF   PTS   SEASON     VALUE  
2  3.5  3.8  1.0  0.2  1.7  2.0  14.1  2018-19  4.472735  
0  3.4  3.4  1.0  0.2  1.7  2.2  15.1  2019-20  4.856631  
3  3.6  3.3  1.1  0.3  1.2  2.3  16.4  2020-21  6.782404  

[3 rows x 31 columns]
         PLAYER POS  AGE TEAM   G  GS    MP   FG   FGA    FG%  ...  DRB   TRB  \
5  Clint Capela   C   24  HOU  67  67  33.6  7.1  10.9  0.648  ...  8.2  12.7   
2  Clint Capela   C   25  HOU  39  39  32.8  6.3   9.9  0.629  ...  9.5  13.8   
6  Clint Capela   C   26  ATL  63  63  30.1  6.6  11.0  0.594  ...  9.6  14.3   

   AST  STL  BLK  TOV   PF   PTS   SEASON     VALUE  
5  1.4  0.7  1.5  1.4  2.5  16.6  2018-19

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a c


       PLAYER POS  AGE TEAM   G  GS      MP     FG    FGA       FG%  ...  \
8  Jeff Green  PF   32  WAS  77  44  27.200  4.200  8.900  0.475000  ...   
3  Jeff Green   C   33  HOU  48   4  19.975  3.325  7.175  0.463415  ...   
9  Jeff Green   C   34  BRK  68  38  27.000  3.800  7.800  0.492000  ...   

      DRB    TRB    AST   STL    BLK     TOV      PF    PTS   SEASON     VALUE  
8  3.3000  4.000  1.800  0.60  0.500  1.3000  2.1000  12.30  2018-19  2.463434  
3  2.2375  2.775  1.075  0.55  0.375  0.8625  1.8375   9.45  2019-20  0.005041  
9  3.4000  3.900  1.600  0.50  0.400  0.8000  1.8000  11.00  2020-21  1.102261  

[3 rows x 31 columns]
       PLAYER POS  AGE TEAM   G  GS    MP   FG   FGA    FG%  ...  DRB  TRB  \
5  Joe Harris  SG   27  BRK  76  76  30.2  4.9   9.8  0.500  ...  3.1  3.8   
2  Joe Harris  SF   28  BRK  69  69  30.8  5.5  11.4  0.486  ...  3.4  4.3   
6  Joe Harris  SF   29  BRK  69  65  31.0  5.2  10.2  0.505  ...  3.0  3.6   

   AST  STL  BLK  TOV   PF   PTS  

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a c


          PLAYER POS  AGE TEAM   G  GS         MP        FG       FGA  \
6  Iman Shumpert  SG   28  HOU  62  41  23.909677  2.716129  7.203226   
3  Iman Shumpert  SF   29  BRK  13   0  18.500000  1.600000  4.900000   
7  Iman Shumpert  SG   30  BRK   2   0   5.500000  0.500000  2.000000   

        FG%  ...       DRB       TRB       AST      STL       BLK       TOV  \
6  0.377071  ...  2.503226  2.970968  1.845161  0.93871  0.403226  0.803226   
3  0.328000  ...  1.900000  2.600000  0.900000  0.90000  0.200000  0.800000   
7  0.250000  ...  0.000000  0.500000  0.000000  0.50000  0.000000  0.500000   

         PF       PTS   SEASON      VALUE  
6  2.070968  7.512903  2018-19   0.898677  
3  1.600000  4.200000  2019-20  -3.136535  
7  0.000000  1.000000  2020-21 -10.716914  

[3 rows x 31 columns]
        PLAYER POS  AGE TEAM   G  GS         MP     FG       FGA       FG%  \
5  Noah Vonleh  PF   23  NYK  68  57  25.300000  3.000  6.500000  0.470000   
2  Noah Vonleh   C   24  DEN  36  

...


        PLAYER POS  AGE TEAM   G  GS         MP        FG        FGA  \
8  J.J. Redick  SG   34  PHI  76  63  31.300000  5.900000  13.500000   
3  J.J. Redick  SG   35  NOP  60  36  26.400000  4.800000  10.500000   
9  J.J. Redick  SG   36  DAL  44   0  16.443182  2.415909   6.072727   

        FG%  ...       DRB       TRB       AST       STL  BLK  TOV        PF  \
8  0.440000  ...  2.200000  2.400000  2.700000  0.400000  0.2  1.3  1.700000   
3  0.453000  ...  2.400000  2.500000  2.000000  0.300000  0.2  1.3  1.600000   
9  0.397829  ...  1.363636  1.463636  1.152273  0.270455  0.1  0.8  1.140909   

         PTS   SEASON     VALUE  
8  18.100000  2018-19  4.758822  
3  15.300000  2019-20  3.171446  
9   7.429545  2020-21 -1.160915  

[3 rows x 31 columns]
         PLAYER POS  AGE TEAM   G  GS         MP        FG       FGA  \
8  JaVale McGee   C   31  LAL  75  62  22.300000  5.300000  8.500000   
3  JaVale McGee   C   32  LAL  68  68  16.600000  2.900000  4.500000   
9  JaVale McGe

        PLAYER POS  AGE TEAM   G  GS         MP        FG      FGA       FG%  \
2  Luke Kornet  PF   23  NYK  46  18  17.000000  2.300000  6.20000  0.378000   
0  Luke Kornet   C   24  CHI  36  14  15.500000  2.300000  5.20000  0.439000   
3  Luke Kornet   C   25  BOS  31   2  11.206452  1.396774  3.26129  0.428289   

   ...      DRB       TRB       AST       STL       BLK       TOV        PF  \
2  ...  2.30000  2.900000  1.200000  0.600000  0.900000  0.500000  0.900000   
0  ...  1.70000  2.300000  0.900000  0.300000  0.700000  0.400000  1.500000   
3  ...  1.73871  2.187097  0.764516  0.141935  1.022581  0.216129  0.680645   

        PTS   SEASON     VALUE  
2  7.000000  2018-19 -0.234091  
0  6.000000  2019-20 -2.427412  
3  3.393548  2020-21 -3.886593  

[3 rows x 31 columns]
             PLAYER POS  AGE TEAM   G  GS         MP        FG       FGA  \
6  Maurice Harkless  SF   25  POR  60  53  23.600000  3.200000  6.500000   
3  Maurice Harkless  PF   26  LAC  62  48  22.993548  2

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a c

                  PLAYER POS  AGE TEAM   G  GS         MP        FG       FGA  \
1  Sviatoslav Mykhailiuk  SF   21  DET  42   0  10.507143  1.071429  3.350000   
0  Sviatoslav Mykhailiuk  SF   22  DET  56  27  22.600000  3.000000  7.200000   
2  Sviatoslav Mykhailiuk  SF   23  DET  66  14  20.054545  3.118182  7.654545   

        FG%  ...       DRB       TRB       AST  STL  BLK       TOV        PF  \
1  0.319829  ...  0.671429  0.885714  0.835714  0.3  0.0  0.485714  0.557143   
0  0.410000  ...  1.600000  1.900000  1.900000  0.7  0.1  1.100000  1.600000   
2  0.407363  ...  2.118182  2.509091  1.690909  0.8  0.2  1.327273  0.945455   

        PTS   SEASON     VALUE  
1  3.207143  2018-19 -5.192421  
0  9.000000  2019-20  1.060706  
2  8.445455  2020-21  0.429285  

[3 rows x 31 columns]
           PLAYER POS  AGE TEAM   G  GS         MP        FG        FGA  \
3  Taurean Prince  SF   24  ATL  55  47  28.200000  4.800000  10.800000   
1  Taurean Prince  PF   25  BRK  64  61  29.00000

/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/jefferywang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a c